
# First-Order Splitting Patterns in <sup>1</sup>H NMR Spectra

## **Instructions:** From the "Cell" dropdown menu, select  "Run All". The interactive plots that accompany the text will be activated.

In [ ]:
from IPython.display import HTML

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The python code for this notebook is hidden by default. 
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

This tutorial assumes the user is familiar with NMR spectroscopy at least up to the meaning of chemical shifts and signal integrations. Familiarity with the "n + 1" rule is helpful, but this will be covered in the tutorial.

In [ ]:
import numpy as np

In [ ]:
import holoviews as hv
import panel as pn
hv.extension('bokeh', width=100)
pn.extension()

In [ ]:
from nmrsim.firstorder import multiplet
from nmrsim.math import add_lorentzians


## What Is "First-Order" Behavior?

To accurately model NMR spectra, quantum mechanical calculations are required. This is not something that can be done "on the back of an envelope" with a calculator and ruler. However, it is often possible to adequately interpret the spectra using a simpler physical model. We will describe and use this model in this tutorial. Spectra that closely follow this simpler model are said to follow "first-order behavior". The explanations that follow use this simplistic physical model.

When the observed spectra cannot adequately be interpreted with this approximate model, they are said to follow "second-order behavior", and more complicated calculations are required to interpret them accurately. We will see an example of this after we introduce the concept of *J*-coupling, starting with the "doublet" pattern.

In [ ]:
def lineshape_from_peaklist(peaklist, w=0.5, points=800, limits=None):
    """

    Parameters
    ----------
    peaklist : [(float, float)...]
        A list of (frequency, intensity) tuples
    w : float
        Peak width at half height (Hz)
    points : int
        The number of data points in the lineshape
    limits : (float, float)
        The frequency (left/right) limits for the lineshape

    Returns
    -------
    ([float...], [float...])
        The lists of x coordinates (frequency) and corresponding y coordinates (intensity) for the lineshape.
    """
    peaklist.sort()
    if limits:
        try:
            l_limit, r_limit = limits
            l_limit = float(l_limit)
            r_limit = float(r_limit)
        except Exception as e:
            print(e)
            print('limits must be a tuple of two numbers')
            raise
        if l_limit > r_limit:
            l_limit, r_limit = r_limit, l_limit
    else:
        l_limit = peaklist[0][0] - 50
        r_limit = peaklist[-1][0] + 50
    x = np.linspace(l_limit, r_limit, points)
    y = add_lorentzians(x, peaklist, w)
    return x, y

In [ ]:
def n_plus_one(J, n, v=100, i = 1, w=0.5):
    singlet = (v, i)  # center at 100 Hz; intensity 1
    couplings = [(J, n)]
    peaklist = multiplet(singlet, couplings)
    x, y = lineshape_from_peaklist(peaklist, w=w)
    return hv.Curve(zip(x, y)).options(axiswise=True, invert_xaxis=True)

## *J* Couplings

A nucleus can "feel" the magnetic moment of a neighboring //adjective nucleus. Spin-1/2 nuclei can be considered to be "spin-up" ("α"; aligned with the spectrometer's magnetic field) or "spin-down" ("β"; aligned against the spectrometer's magnetic field). The odds of a neighboring spin-1/2 nucleus being either spin-up or spin-down are very nearly equal. 

The small magnetic moment of neighboring nuclei affects the net magnetic field strengh that a nucleus experiences. If a nucleus has one such neighbor, there's close to a 50% chance that it will experience a slightly stronger total magnetic field, and a 50% chance of a slightly weaker magnetic field. That means that half the time the nucleus will resonate at a slightly higher frequency, and half the time at a slightly lower frequency. The two nuclei are said to be "coupled", and the size of the frequency difference between the high- and low-frequency peaks in Hz is referred to as the "coupling constant", or *J*-value.

You can simulate this behavior using the plot below. In the absence of *J*-coupling, a single peak or "singlet" is seen. Check the box to toggle the *J* coupling to one neighbor on and off. Turning the coupling on changes the singlet to a two-peak "doublet".



In [ ]:
def toggle_doublet(coupled=False):
    n = 1 if coupled else 0
    return n_plus_one(10.0, n, w=0.5).redim(y=hv.Dimension('y', range=(-0.1, 1.1)))


toggle_doublet_app = pn.interact(toggle_doublet, n=False)

toggle_doublet_app

Note that when the coupling is turned on, the intensities of the two peaks of the resulting doublet are half that of the original singlet. 
The signal appears as a singlet at 100 Hz in the absence of coupling, but two signals at 95 and 105 Hz in the presence of a 10-Hz *J* coupling.

The simulations in this tutorial will use Hz as the x-axis. A signal at 100 Hz would appear at 100 Hz higher frequency than the signal for TMS (tetramethylsilane). In standard NMR spectra, the ppm (parts per million) scale is used instead. For <sup>1</sup>H NMR spectra, if you divide the frequency of the signal in Hz by the frequency of the spectrometer, you get the chemical shift in ppm. This 100-Hz signal would be 1 ppm on a 100 MHz spectrometer, but 0.25 ppm on a 400 MHz spectrometer (the standard spectrometer we use for CHEM333 lab samples).

In [ ]:
from nmrsim.discrete import AB

In [ ]:
def ab_wrapper(v1, v2, J):
    vab = abs(v2 - v1)
    vcentr = (v1 + v2) / 2
    peaklist = AB(J, vab, vcentr)
    return peaklist

In [ ]:
def ab_distortion(v1 = 100, v2 = 200):
#     singlet_1 = multiplet((v1, 1), [(10, 0)])
#     singlet_2 = multiplet((v2, 1), [(10, 0)])
#     doublet_1 = multiplet((v1, 1), [(10, 1)])
#     doublet_2 = multiplet((v2, 1), [(10, 1)])
    v1_arrow = hv.Arrow(v1,0, 'v1', '^')
    v2_arrow = hv.Arrow(v2,0, 'v2', '^')
    datapoints = max(int(abs(v2 - v1) * 100), 800)  # sufficient datapoints to mitigate inaccurate intensities and "jittering"

    coupled = lineshape_from_peaklist(ab_wrapper(v1, v2, 10), points=datapoints)
    return hv.Curve(zip(*coupled)) * v1_arrow * v2_arrow

In [ ]:
ab_distortion_interactive = pn.interact(ab_distortion, v1=(50, 350, 0.1, 50), v2=(50, 350, 0.1, 350))

In [ ]:
ab_distortion_interactive

## Origin of the "n + 1" Rule

A neighboring spin-1/2 nuclei (e.g <sup>1</sup>H; <sup>13</sup>C) nucleus has close to a 50:50 chance of being spin-up vs. spin-down, with each state adding or subtracting to the overall strength of the magnetic field experienced by the nucleus of interest. The statistical chance of the nucleus of interest experiencing a certain magnetic field strength (and thus a certain frequency/chemical shift) resembles the statistical chance of head/tails coin flips. If you flip two coins, the possible results are:

* both heads: 25%
* one heads, one tails: 50%
* both tails: 25%

or 1 : 2 : 1. 

Similarly, if a proton has two proton neighbors, each of which can be either α or β, the odds are close to:

* both α: 25%
* one α, one β: 50 %
* both β: 25%

OR 1 : 2 : 1. 

For the plot below, drag the slider to change the number of neighbors n (each with a 10-Hz coupling) from 0 to 1 to 2. When n = 2, a "triplet" pattern with 1 : 2 : 1 intensities results. Compared to the intensity of the singlet in the absence of coupling, the triplet intensities are 0.25 : 0.5 : 0.25. Note that the integration of this signal will be 1 regardless of the number of splittings.

In [ ]:
def singlet_to_triplet(n):
    return n_plus_one(J=10.0, n=n, w=0.5).redim(y=hv.Dimension('y', range=(-0.1, 1.1)))

In [ ]:
singlet_to_triplet_app = pn.interact(singlet_to_triplet, n=(0, 2, 1, 0))

In [ ]:
singlet_to_triplet_app

In [ ]:
def simple_multiplet(n):
    return n_plus_one(J=7.0, n=n, w=0.5).redim(y=hv.Dimension('y', range=(-0.1, 1.1)))

simple_multiplet_app = pn.interact(simple_multiplet, n=(0, 9, 1, 1))


In [ ]:
def toggle_doublet(coupled=False):
    n = 1 if coupled else 0
    return n_plus_one(10.0, n, w=0.5).redim(y=hv.Dimension('y', range=(-0.1, 1.1)))


toggle_doublet_app = pn.interact(toggle_doublet, n=False)

toggle_doublet_app

Note that when the coupling is turned on, the intensities of the two peaks of the resulting doublet are half that of the original singlet. 
The signal appears as a singlet at 100 Hz in the absence of coupling, but two signals at 95 and 105 Hz in the presence of a 10-Hz *J* coupling.

The simulations in this tutorial will use Hz as the x-axis. A signal at 100 Hz would appear at 100 Hz higher frequency than the signal for TMS (tetramethylsilane). In standard NMR spectra, the ppm (parts per million) scale is used instead. For <sup>1</sup>H NMR spectra, if you divide the frequency of the signal in Hz by the frequency of the spectrometer, you get the chemical shift in ppm. This 100-Hz signal would be 1 ppm on a 100 MHz spectrometer, but 0.25 ppm on a 400 MHz spectrometer (the standard spectrometer we use for CHEM333 lab samples).

In [ ]:
from nmrsim.discrete import AB

In [ ]:
def ab_wrapper(v1, v2, J):
    vab = abs(v2 - v1)
    vcentr = (v1 + v2) / 2
    peaklist = AB(J, vab, vcentr)
    return peaklist

In [ ]:
def ab_distortion(v1 = 100, v2 = 200):
#     singlet_1 = multiplet((v1, 1), [(10, 0)])
#     singlet_2 = multiplet((v2, 1), [(10, 0)])
#     doublet_1 = multiplet((v1, 1), [(10, 1)])
#     doublet_2 = multiplet((v2, 1), [(10, 1)])
    v1_arrow = hv.Arrow(v1,0, 'v1', '^')
    v2_arrow = hv.Arrow(v2,0, 'v2', '^')
    datapoints = max(int(abs(v2 - v1) * 100), 800)  # sufficient datapoints to mitigate inaccurate intensities and "jittering"

    coupled = lineshape_from_peaklist(ab_wrapper(v1, v2, 10), points=datapoints)
    return hv.Curve(zip(*coupled)) * v1_arrow * v2_arrow

In [ ]:
ab_distortion_interactive = pn.interact(ab_distortion, v1=(50, 350, 0.1, 50), v2=(50, 350, 0.1, 350))

In [ ]:
ab_distortion_interactive

## Origin of the "n + 1" Rule

A neighboring spin-1/2 nuclei (e.g <sup>1</sup>H; <sup>13</sup>C) nucleus has close to a 50:50 chance of being spin-up vs. spin-down, with each state adding or subtracting to the overall strength of the magnetic field experienced by the nucleus of interest. The statistical chance of the nucleus of interest experiencing a certain magnetic field strength (and thus a certain frequency/chemical shift) resembles the statistical chance of head/tails coin flips. If you flip two coins, the possible results are:

* both heads: 25%
* one heads, one tails: 50%
* both tails: 25%

or 1 : 2 : 1. 

Similarly, if a proton has two proton neighbors, each of which can be either α or β, the odds are close to:

* both α: 25%
* one α, one β: 50 %
* both β: 25%

OR 1 : 2 : 1. 

For the plot below, drag the slider to change the number of neighbors n (each with a 10-Hz coupling) from 0 to 1 to 2. When n = 2, a "triplet" pattern with 1 : 2 : 1 intensities results. Compared to the intensity of the singlet in the absence of coupling, the triplet intensities are 0.25 : 0.5 : 0.25. Note that the integration of this signal will be 1 regardless of the number of splittings.

In [ ]:
def singlet_to_triplet(n):
    return n_plus_one(J=10.0, n=n, w=0.5).redim(y=hv.Dimension('y', range=(-0.1, 1.1)))

In [ ]:
singlet_to_triplet_app = pn.interact(singlet_to_triplet, n=(0, 2, 1, 0))

In [ ]:
singlet_to_triplet_app

In [ ]:
def simple_multiplet(n):
    return n_plus_one(J=7.0, n=n, w=0.5).redim(y=hv.Dimension('y', range=(-0.1, 1.1)))

In [ ]:
simple_multiplet_app = pn.interact(simple_multiplet, n=(0, 9, 1, 1))


In [ ]:
simple_multiplet_row = pn.Row(simple_multiplet_app, width_policy='max')


In [ ]:
simple_multiplet_row